In [3]:
! pip install langchain_community
! pip install langchain_text_splitter
! pip install langchain_openai
! pip install faiss-cpu
! pip install pymupdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.2 which is incompatible.



  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.6-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   ---------------------------------------- 883.8/883.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.108
    Uninstalling langsmith-0.1.108:
      Successfully uninstalled langsmith-0.1.108
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.37
    Uninstalling langchain-core-0.2.37:
      Successfully uninstalled langchain-core-0.2.37


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.15 requires langchain-core<0.3.0,>=0.2.35, but you have langchain-core 0.3.10 which is incompatible.
langchain-community 0.2.15 requires langchain-core<0.3.0,>=0.2.37, but you have langchain-core 0.3.10 which is incompatible.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.3.10 which is incompatible.


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.9 MB 5.3 MB/s eta 0:00:03
   ---------- ----------------------------- 3.9/14.9 MB 6.3 MB/s eta 0:00:02
   --------------- ------------------------ 5.8/14.9 MB 7.2 MB/s eta 0:00:02
   --------------------- ------------------ 7.9/14.9 MB 7.6 MB/s eta 0:00:01
   -------------------------- ------------- 9.7/14.9 MB 7.9 MB/s eta 0:00:01
   -------------------------------- ------- 12.1/14.9 MB 8.3 MB/s eta 0:00:01
   ------------------------------------- -- 13.9/14.9 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 8.1 MB/s eta 0:00:00


In [16]:
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()

True

In [17]:
import os

current_path = os.getcwd()
print("Current path:", current_path)


Current path: c:\Users\carpe\OneDrive\Desktop\p\rag


In [ ]:
#pip install langchain==0.0.208
#랭체인 버전을 낮춤

In [25]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings  # OpenAIEmbeddings 경로 수정
from langchain.document_loaders import PyPDFLoader  # PyMuPDFLoader 대신 PyPDFLoader 사용

# 파서 직접 구현
class CustomOutputParser:
    def parse(self, llm_output: str) -> str:
        return llm_output

# RunnablePassthrough 대체 함수
def passthrough(value):
    return value

# CSV 파일 로드
loader = CSVLoader(file_path="test03.csv", encoding='utf-8')

data = loader.load()

print(f"Loaded {len(data)} documents from the csv.")

# print(data)  # 필요에 따라 주석 해제


ImportError: cannot import name 'OpenAIEmbeddings' from 'langchain.embeddings.openai' (c:\Users\carpe\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\embeddings\openai.py)

In [ ]:
# 위에서 불러온 csv 데이터 청크
# RecursiveCharacterTextSplitter를 사용
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(data)
len(splits)

# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

vectorstore.save_local('./db/faiss')

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "30분 이내에 만들 수 있는 중식 요리 방법 알려줘"
answer = rag_chain.invoke(question)
print(answer)